In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
LINK = "https://www.moneycontrol.com/news/tags/hdfc.html/news/"

In [3]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Open LINK
driver.get(LINK)

In [4]:
def get_page_number(link):
    return int(link.split("/")[-2].split("-")[-1])

def get_page_link(num, link=LINK):
    return f"{link}page-{num}/"

In [5]:
last_page_button = driver.find_element(
    By.CLASS_NAME, "pagenation").find_elements(
        By.CLASS_NAME, "last"
    )[-1]

In [6]:
total_pages = get_page_number(last_page_button.get_attribute("href"))

In [7]:
def get_articles_div(page):
    return page.find_element(
        By.CLASS_NAME, "topictabpane").find_elements(By.CLASS_NAME, "clearfix")

def get_article_metadata(article, article_num):
    try:
        return (
            article.find_element(
                By.CLASS_NAME, "lazy-container").text, 
            article.find_element(
                By.CLASS_NAME, "lazy-container").find_element(By.TAG_NAME, "a").get_attribute("href")
            )
    except:
        
        return (
            article.find_element(
                By.TAG_NAME, "a").find_element(By.XPATH, f'//*[@id="newslist-{article_num}"]/span').text,
            article.find_element(
                By.TAG_NAME, "a").get_attribute("href")
        )

In [9]:
data = []
for page_num in range(1, total_pages+1):
    page_link = get_page_link(page_num, LINK)
    driver.get(page_link)
    articles = get_articles_div(driver)
    for i, article in enumerate(articles):
        data.append(get_article_metadata(article, i))
    

In [32]:
data = pd.DataFrame(data)
data.columns = ["Date", "link"]
data.to_csv("checkpoint.csv", index=False)

In [16]:
from newspaper import Article
from tqdm.notebook import tqdm

In [20]:
main_data = []
failed = []
for link in tqdm(data[1].to_list()):
    try:
        article = Article(link)
        article.download()
        article.parse()
        main_data.append({'link': link, 'text': article.text})
    except:
        print(link)
        failed.append(link)

  0%|          | 0/749 [00:00<?, ?it/s]

https://www.moneycontrol.com/news/india/moneycontrol-selects-top-stories-this-afternoon-134-11168701.html
https://www.moneycontrol.com/news/business/markets/hdfc-says-july-13-delisting-date-is-tentative-10867861.html
https://www.moneycontrol.com/news/business/hdfcs-exit-from-corporate-bond-market-brings-cheer-to-smaller-issuers-as-borrowing-costs-fall-10866981.html
https://www.moneycontrol.com/news/business/expecting-regulatory-approval-within-a-month-for-credila-sale-parekh-10866001.html


In [23]:
len(failed)

4

In [24]:
retry_data = []
failed_again = []
for link in tqdm(failed):
    try:
        article = Article(link)
        article.download()
        article.parse()
        retry_data.append({'link': link, 'text': article.text})
    except:
        print(link)
        failed_again.append(link)

  0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
main_data.extend(retry_data)

In [30]:
# Create a pandas DataFrame from the data list
df = pd.DataFrame(main_data)

In [38]:
data = data.merge(df)

In [39]:
data.to_csv('extracted_data.csv', index=False)

print("Data extraction and saving complete.")

Data extraction and saving complete.


In [40]:
driver.quit()